In [1]:
import os, sys
!{sys.executable} -m pip install -r requirements.txt

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
potara 1.0.2 requires networkx==1.8.1, but you have networkx 2.5 which is incompatible.
potara 1.0.2 requires nltk==3.4.5, but you have nltk 3.5 which is incompatible.
potara 1.0.2 requires six==1.12.0, but you have six 1.15.0 which is incompatible.
en-core-web-md 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 2.1.0 which is incompatible.
You should consider upgrading via the 'C:\Users\Stefa\.conda\envs\python36\python.exe -m pip install --upgrade pip' command.



  Using cached boto3-1.17.62-py2.py3-none-any.whl (131 kB)
  Using cached botocore-1.20.62-py2.py3-none-any.whl (7.5 MB)
  Using cached feedparser-6.0.2-py3-none-any.whl (80 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached lxml-4.6.3-cp36-cp36m-win_amd64.whl (3.5 MB)
  Using cached neo4j-4.2.1.tar.gz (69 kB)
  Using cached nltk-3.5-py3-none-any.whl
  Using cached python_dotenv-0.17.1-py2.py3-none-any.whl (18 kB)
  Using cached PyYAML-5.4.1-cp36-cp36m-win_amd64.whl (209 kB)
  Using cached regex-2021.3.17-cp36-cp36m-win_amd64.whl (270 kB)
  Using cached s3transfer-0.4.2-py2.py3-none-any.whl (79 kB)
  Using cached soupsieve-2.2.1-py3-none-any.whl (33 kB)


  Using cached tqdm-4.59.0-py2.py3-none-any.whl (74 kB)
  Using cached urllib3-1.26.4-py2.py3-none-any.whl (153 kB)
  Created wheel for neo4j: filename=neo4j-4.2.1-py3-none-any.whl size=95191 sha256=b771726e6fea86ed5a1774fbb58cac963ee8daed09a01b0481c2ac96eceea02b
  Stored in directory: c:\users\stefa\appdata\local\pip\cache\wheels\e5\2f\1a\6500122f1df4f7a2f15041cd9e2102a4fb225ef8ce8516448a
Successfully built neo4j
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.48.2
    Uninstalling tqdm-4.48.2:
      Successfully uninstalled tqdm-4.48.2
  Attempting uninstall: soupsieve
    Found existing installation: soupsieve 2.0.1
    Uninstalling soupsieve-2.0.1:
      Successfully uninstalled soupsieve-2.0.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.53
    Uninstalling botocore-

In [2]:
from neo4j import GraphDatabase
from dotenv import dotenv_values
from soup_models import Document
from collections import defaultdict

config = dotenv_values(".env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}

C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [49]:
class GraphDBDriver:
    """
    Main methods:
        query_node_dict: Query database by a node dictionary
        query: makes a direct cypher query
        upload_nodes: Upload an iterable of nodes to the database

    """
    def __init__(self, uri=config["LOCAL_GRAPH_URI"], user=config["LOCAL_GRAPH_USER"], password=config["LOCAL_GRAPH_PWD"]):
        try:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
        except Exception as e:
            print("Failed to create the driver:", e)
        # self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver:
            self.driver.close()

    # def upload_test(self):
    #     assert self.driver, "Driver not initialized!"
    #     with self.driver.session() as session:
    #         return session.write_transaction(self._create_node, self._node_dict_to_cypher({'type': "Document", 'title': "test1"}))

    # Query Methods
    def query_node_dict(self, node_dict):
        return self.query("MATCH " + self._node_dict_to_cypher(node_dict) + " RETURN node")
    
    def query_node(self, node):
#         print("MATCH (node:{} {{key: \"{}\"}}) RETURN node".format(node.type, node.key))
        return self.query("MATCH (node:{} {{key: \"{}\"}}) RETURN node".format(node.type, node.key))

    def query(self, query, db=None):
        assert self.driver, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.driver.session(database=db) if db is not None else self.driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session:
                session.close()
        return response

    # Upload Methods
    def upload_nodes(self, nodes):
        assert self.driver, "Driver not initialized!"
        with self.driver.session() as session:
            ret = []
            for node in nodes:
                exists = list(session.run("MATCH (node:{} {{key: \"{}\"}}) RETURN node".format(node.type, node.key)))
                if len(exists) == 0:
                    # assert type(doc) == Document , "Error: non-Document node passed to doc upload function"
                    ret.append(session.write_transaction(self._create_and_return_node, node.to_dict()))
                    print("Uploaded", ret[-1])
                else:
                    print(node.key, "already exists in database")
            return ret

    @staticmethod
    def _create_and_return_node(tx, node_dict):
        cypherquery = ["CREATE (node:{})".format(node_dict['type'])]
        for key in node_dict.keys():
            cypherquery.append("SET node.{} = ${}".format(key, key))
        cypherquery.append("RETURN node.title + ' at ID:' + id(node)")
#         print(" ".join(cypherquery))
        result = tx.run(" ".join(cypherquery), node_dict)
        return result.single()[0]

    # Helper Methods
    """
    Converts a node in dictionary form to a cypher create query
    """
    @staticmethod
    def _node_dict_to_cypher(node, name="node"):
        assert type(node) in [dict, defaultdict]
        query = "({}:{} {{".format(name, node['type'])
        properties = []
        for key, value in node.items():
            properties.append("{}:\"{}\"".format(key, value))
        props = ", ".join(properties)
        end_query = "})"
        return query + props + end_query

In [50]:
# greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "neo4j")
driver = GraphDBDriver(config["LOCAL_GRAPH_URI"], config["LOCAL_GRAPH_USER"], config["LOCAL_GRAPH_PWD"])
# driver.upload_doc("hello, world")
# driver.upload_docs([{'type': "document", 'title': "test1"}])
# print(driver.query("MATCH (n) return n"))
# print(driver.query_node_dict({'type': "document", 'title': "test1"}))
# print(driver._node_dict_to_cypher({'type': "document", 'title': "test1"}))
# print(driver.upload_test())
# driver.close()

In [51]:
tweets = [Document("test1", "title", "tweet"), Document("test2", "title", "tweet"), Document("test3", "title", "tweet")]
# tweet.attrs["details"] = "hello!"
# driver.upload_nodes([tweet])

In [52]:
nodes = driver.query("MATCH (n) RETURN n")

In [53]:
for node_type in ["document", "entity", "action", "source"]:
    querystring = "CREATE CONSTRAINT unique_key_{} IF NOT EXISTS ON (n:{}) ASSERT n.key IS UNIQUE".format(node_type, node_type)
    print(driver.query(querystring))

[]
[]
[]
[]


In [54]:
driver.query_node(tweet)

[<Record node=<Node id=16 labels=frozenset({'document'}) properties={'details': 'hello!', 'doc_type': 'tweet', 'type': 'document', 'date_processed': neo4j.time.DateTime(2021, 5, 22, 16, 42, 13.227833), 'title': 'title', 'key': 'test1'}>>]

In [55]:
driver.upload_nodes(tweets)

test1 already exists in database
test2 already exists in database
test3 already exists in database


[]